In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

In [2]:
torch.manual_seed(2020)
np.random.seed(2020)

In [3]:
{'lr': 0.0012098123619624396,
 'layers': 2,
 'step_size': 21,
 'gamma': 0.5302067528042456,
 'bptt': 12,
 'dropout': 0.35583243487203325}

{'lr': 0.0012098123619624396,
 'layers': 2,
 'step_size': 21,
 'gamma': 0.5302067528042456,
 'bptt': 12,
 'dropout': 0.35583243487203325}

In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [5]:
dataset = pd.read_csv('/home/urwa/Documents/side_projects/urban/data/featureData/lga.csv')

In [6]:
dataset.shape

(8757, 1045)

In [7]:
dataset.head(3)

,Date,Hour,1,10,100,101,102,106,107,108,...,91_lag_3,92_lag_3,93_lag_3,94_lag_3,95_lag_3,96_lag_3,97_lag_3,98_lag_3,99_lag_3,arrival_lag_3
0,2018-01-01,3,0,0,0,0,0,0,0,0,...,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,3.0
1,2018-01-01,4,1,0,0,0,0,0,0,0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,2018-01-01,5,1,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [8]:
lag_columns = [c for c in dataset.columns if 'lag' in c]
len(lag_columns)

774

In [9]:
dataset = dataset[[c for c in dataset.columns if c not in lag_columns]]
dataset.shape

(8757, 271)

In [10]:
DateColumns = ['Date']

ext_columns = ['Dow', 'arrival','maxtemp', 'mintemp', 'avgtemp', 'departure', 'hdd',
       'cdd', 'participation', 'newsnow', 'snowdepth', 'ifSnow']

targetColumns = [c for c in dataset.columns if c not in ext_columns and \
                c not in DateColumns and c not in lag_columns and c != 'Hour']
len(targetColumns)

257

In [11]:
features_cols = [c for c in dataset.columns if c not in targetColumns and c not in DateColumns]
len(features_cols)

13

In [12]:
sep = int(0.75*len(dataset))
print(sep)


trainData = dataset[:sep]
testData = dataset[sep:]

print(trainData.shape)
print(testData.shape)

6567
(6567, 271)
(2190, 271)


In [13]:
X_train = trainData[features_cols].values
X_train = torch.tensor(X_train).float().to(device)
print(X_train.shape)

y_train = trainData[targetColumns].values
y_train = torch.tensor(y_train).float().to(device)
print(y_train.shape)

X_test = testData[features_cols].values
X_test = torch.tensor(X_test).float().to(device)
print(X_test.shape)

y_test = testData[targetColumns].values
y_test = torch.tensor(y_test).float().to(device)
print(y_test.shape)

torch.Size([6567, 13])
torch.Size([6567, 257])
torch.Size([2190, 13])
torch.Size([2190, 257])


In [14]:
def create_inout_sequences(x,y, tw):
    inout_seq = []
    L = len(x)
    for i in range(L-tw):
        train_seq_x = x[i:i+tw]
        train_seq_y = y[i:i+tw]
#         train_seq = torch.cat((train_seq_x,train_seq_y),axis=1)
        
#         train_label = y[i+tw:i+tw+1]
        train_label = y[i+1:i+tw+1]
        inout_seq.append((train_seq_x, train_seq_y ,train_label))
    return inout_seq

In [15]:
bptt = 12

In [16]:
train_inout_seq = create_inout_sequences(X_train,y_train, bptt)

In [17]:
train_inout_seq[0][0].shape,train_inout_seq[0][1].shape, train_inout_seq[0][2].shape

(torch.Size([12, 13]), torch.Size([12, 257]), torch.Size([12, 257]))

In [18]:
test_inout_seq = create_inout_sequences(X_test,y_test, bptt)

In [19]:
class LSTM(nn.Module):
    def __init__(self, feat_size=1, hidden_layer_size=100, network_size=1, layers=1, communities=10, dropout=0, at_mat=None):
        super().__init__()
        
        # aggregation
        if at_mat != None:
            self.attachment_matrix = torch.nn.Parameter(at_mat)
            self.attachment_matrix.requires_grad = False
        else:
            self.attachment_matrix = torch.nn.Parameter(torch.randn(network_size,communities))
            self.attachment_matrix.requires_grad = True
        
        
        self.hidden_layer_size = hidden_layer_size
        
        self.hidden_cell = (torch.zeros(layers,1,self.hidden_layer_size),
                    torch.zeros(layers,1,self.hidden_layer_size))
        
        lstm_input = communities + feat_size
        self.lstm = nn.LSTM(input_size=lstm_input, hidden_size=hidden_layer_size, num_layers=layers, dropout=dropout)

        #disaggregation
#         self.linear_1 = nn.Linear(hidden_layer_size, hidden_layer_size)
        self.linear_2 = nn.Linear(hidden_layer_size, network_size)


    def forward(self, input_seq, feat):
        
        w = F.softmax(self.attachment_matrix, dim=1)
        x = torch.matmul(input_seq, self.attachment_matrix)
        x = torch.cat((x,feat),axis=1)

        
        lstm_out, self.hidden_cell = self.lstm(x.view(len(input_seq) ,1, -1), self.hidden_cell)
        
        predictions = self.linear_2(lstm_out.view(len(input_seq), -1))
#         predictions = F.relu(predictions)
#         predictions = self.linear_2(predictions)
        
        return predictions

In [20]:
def evaluate(model):
    model.eval()
    prediction = []
    with torch.no_grad():
        for feat,seq, labels in test_inout_seq:
            model.hidden = (torch.zeros(layers, 1, model.hidden_layer_size),
                            torch.zeros(layers, 1, model.hidden_layer_size))
            prediction.append(model(seq,feat)[-1])

    y_test_ = torch.stack([labels[-1] for feat,seq, labels in test_inout_seq], axis=0).detach().cpu().numpy()
    y_pred_ = torch.stack(prediction).detach().cpu().numpy()

    r2 = r2_score(y_test_, y_pred_, multioutput='variance_weighted')
    rmse = mean_squared_error(y_test_, y_pred_)
    mae = mean_absolute_error(y_test_, y_pred_)
#     print("r2: ",r2)
    return (r2, rmse, mae)

In [21]:
def get_at_mat(targetColumns):
    comms = pd.read_csv('/home/urwa/Documents/side_projects/urban/UrbanTemporalNetworks/Data/ZonetoComm.csv')  
    communities = list(set(comms.start_community))

    mapping = dict(zip(comms.start_id, comms.start_community))
    comm_to_index = dict(zip(communities,range(len(communities))))
    col_to_index = dict(zip(targetColumns,range(len(targetColumns))))

    attach = torch.zeros(len(targetColumns), len(communities))

    for t_c in targetColumns:
        com = mapping[int(t_c)]
        x_i = col_to_index[t_c]
        y_i = comm_to_index[com]

        attach[x_i,y_i] = 1

    return attach

In [22]:
at_mat = get_at_mat(targetColumns)
at_mat.shape

torch.Size([257, 24])

In [23]:
layers = 2
communities = 24
network_size = len(targetColumns)
feat_size = len(features_cols)
dropout=0.35583243487203325

model = LSTM(feat_size = feat_size, hidden_layer_size=communities,
             network_size=network_size, layers=layers,
            communities=communities, dropout=dropout, at_mat=at_mat).to(device)

loss_function = nn.L1Loss()   
optimizer = torch.optim.Adam(model.parameters(), lr=0.0012098123619624396)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=21, gamma=0.530)

In [24]:
epochs = 100
best_r2 = 0

for i in range(epochs):
    model.train()
    for feat,seq, labels in train_inout_seq:
        optimizer.zero_grad()
        model.hidden_cell = (torch.zeros(layers, 1, model.hidden_layer_size).to(device),
                        torch.zeros(layers, 1, model.hidden_layer_size).to(device))

        y_pred = model(seq, feat)

        single_loss = loss_function(y_pred, labels)
        single_loss.backward()
        optimizer.step()
        
    scheduler.step()
#     if i%10 == 1:
    r2, rmse, mae = evaluate(model)
    print(f'epoch: {i:3} loss: {single_loss.item():10.8f} r2: {r2:5.3f} rmse: {rmse:5.3f} mae: {mae:5.3f}')

    if r2 > best_r2:
        best_r2 = r2
        torch.save(model.state_dict(), 'lga.pt')

print(f'epoch: {i:3} loss: {single_loss.item():10.10f}')
print("bet_r2: ", best_r2)

epoch:   0 loss: 1.61770117 r2: 0.570 rmse: 10.013 mae: 1.224
epoch:   1 loss: 1.48797011 r2: 0.646 rmse: 8.258 mae: 1.157
epoch:   2 loss: 1.39927602 r2: 0.681 rmse: 7.435 mae: 1.117
epoch:   3 loss: 1.36847055 r2: 0.680 rmse: 7.470 mae: 1.122
epoch:   4 loss: 1.29878771 r2: 0.693 rmse: 7.162 mae: 1.115
epoch:   5 loss: 1.28894806 r2: 0.712 rmse: 6.715 mae: 1.076
epoch:   6 loss: 1.32515252 r2: 0.711 rmse: 6.732 mae: 1.087
epoch:   7 loss: 1.28571153 r2: 0.702 rmse: 6.945 mae: 1.087
epoch:   8 loss: 1.31155205 r2: 0.726 rmse: 6.384 mae: 1.057
epoch:   9 loss: 1.23294592 r2: 0.725 rmse: 6.417 mae: 1.057
epoch:  10 loss: 1.23393464 r2: 0.722 rmse: 6.479 mae: 1.070
epoch:  11 loss: 1.21626782 r2: 0.731 rmse: 6.261 mae: 1.057
epoch:  12 loss: 1.19701838 r2: 0.740 rmse: 6.054 mae: 1.040
epoch:  13 loss: 1.46755695 r2: 0.732 rmse: 6.237 mae: 1.038
epoch:  14 loss: 1.24417651 r2: 0.719 rmse: 6.545 mae: 1.061
epoch:  15 loss: 1.26248682 r2: 0.733 rmse: 6.224 mae: 1.046
epoch:  16 loss: 1.2309

In [25]:
evaluate(model)

(0.7514492479226754, 5.7937226, 1.0082139)

In [26]:
# attachment = torch.argmax(F.softmax(model.attachment_matrix, dim=1), dim=1).detach().cpu().numpy()
# community_assignment = dict(zip(targetColumns, attachment))
# community_assignment

In [27]:
# community_assignment

In [28]:
attachment = model.attachment_matrix.detach().cpu().numpy()

In [29]:
attachment.shape

(257, 24)

In [30]:
len(targetColumns)

257

In [31]:
from sklearn.cluster import KMeans
from sklearn.metrics import davies_bouldin_score

In [32]:
for i in range(2,24):
    print('========== '+str(i)+' ============')
    kmeans = KMeans(n_clusters=i, random_state=1).fit(attachment)
    labels = kmeans.labels_
    print(davies_bouldin_score(attachment, labels))

========== 2 ============
0.9431580894298828
========== 3 ============
0.9409462285531621
========== 4 ============
0.938227413946092
========== 5 ============
0.9361745238003149
========== 6 ============
0.9336492828481276
========== 7 ============
0.9310861885154232
========== 8 ============
0.9277310771799258
========== 9 ============
0.9240067409725623
========== 10 ============
0.9210297483903906
========== 11 ============
0.9148244938307323
========== 12 ============
0.9106162890704091
========== 13 ============
0.9028384366793064
========== 14 ============
0.8928296466001854
========== 15 ============
0.8862701759183564
========== 16 ============
0.8754781390409807
========== 17 ============
0.8674057593158342
========== 18 ============
0.8517420124209563
========== 19 ============
0.8254840255935164
========== 20 ============
0.7926951398719686
========== 21 ============
0.7372305588529849
========== 22 ============
0.6382612661227811
========== 23 ============
0.34914856479200

In [33]:
labels = kmeans.labels_

In [34]:
labels

array([ 9,  2,  9,  2, 10,  7,  6,  1, 16,  1,  7, 14,  6,  6, 19, 13, 17,
       16,  8,  6, 13,  3,  2,  1,  2,  6,  8, 13, 13, 10,  6,  2,  2,  2,
        1,  2,  3, 11,  9,  2,  1, 18, 18, 15, 15,  6, 12, 12,  8,  6,  1,
        3,  1, 15, 13, 11,  1,  1, 21, 10,  6,  8,  3, 10,  9,  9,  9,  9,
        1, 15,  8,  8,  8, 14,  9,  3, 22, 10, 11,  3, 16,  4,  1, 12, 11,
        2,  7,  5,  5,  5,  5,  9, 21,  4,  7,  2,  7,  2,  3, 12, 20,  7,
       10,  2, 14,  3,  2, 11, 11, 17, 12,  2, 16,  2, 19,  0,  5,  6,  1,
        1,  6,  5,  5, 22,  2,  2, 14,  2,  2,  1, 11, 19,  4, 12,  6, 14,
       12,  1,  1,  9, 21,  9,  6,  6,  9,  0,  8, 18, 18, 15, 15, 15, 11,
       11,  5, 13, 13, 19,  0,  8,  5,  6,  7,  5, 21,  3,  3,  5, 14, 14,
        7,  2,  5,  1, 10,  6, 18, 18, 17,  2,  1,  5, 17,  5,  5,  7,  7,
        4, 14, 14,  2,  4,  6,  7, 20, 13, 15, 16,  6,  5,  8,  9,  7, 16,
        9,  5,  7,  3,  7,  1, 10,  5,  8, 22,  8,  4,  4,  4,  3,  7,  7,
        1,  0,  8, 12, 10

In [35]:
community_assignment = dict(zip(targetColumns, labels.astype(str)))

In [36]:
import json

with open('lga_single.json', 'w') as fp:
    json.dump(community_assignment, fp)

In [37]:
# 20 comm
# 0.505

In [38]:
# 50 comm
# 